In [2]:
import numpy as np, pandas as pd 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import warnings
import copy
from itertools import product
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 1000)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = [u'SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [3]:
matrl_class0 = pd.read_excel('matrl_class.xlsx', sheet_name=0)
matrl_class1 = pd.read_excel('matrl_class.xlsx', sheet_name=1)
matrl_class1['备注'] = np.nan
# matrl_class0['sheet_name'] = 0
# matrl_class1['sheet_name'] = 1

In [4]:
len(matrl_class1)

6641

In [5]:
# 复制一份，方便操作
matrl_class0_copy = copy.deepcopy(matrl_class0)
matrl_class1_copy = copy.deepcopy(matrl_class1)

In [6]:
# 先输出完全重复数据，再输出物料号重复的数据
l1 = matrl_class1_copy.drop_duplicates(subset=['物料', '物料类别', '设备类型'], keep='last', inplace=False)
l2 = matrl_class1_copy.drop_duplicates(subset=['物料', '物料类别', '设备类型'], keep=False, inplace=False)
list_duplicates1 = set(l1['物料']) - set(l2['物料'])
print(len(list_duplicates1), set(list_duplicates1))

matrl_class1_copy.drop_duplicates(subset=['物料', '物料类别', '设备类型'], keep='last', inplace=True)
matrl_class1_copy.reset_index(drop=True, inplace=True)

l1 = matrl_class1_copy.drop_duplicates(subset = ['物料'], keep='last', inplace=False)
l2 = matrl_class1_copy.drop_duplicates(subset = ['物料'], keep=False, inplace=False)
list_duplicates2 = set(l1['物料']) - set(l2['物料'])
print(len(list_duplicates2), set(list_duplicates2))

281 {'00663205400201030', '00663006232400000', '00663400100450000', '00663802600001020', '00663600102401040', '00663406600001011', '00663022100001021', '00663804200430000', '00663303101201030', '00662218400700000', '00663022110200000', '0066GQ20182985001', '1039804643', '00663305400220002', '00663004400001010', '00663800103001020', '00663407600001060', '00663305400230001', '00663309600001011', '00663028200001020', '00663010100401150', '00663002100001021', '00662408505150000', '00660311800001080', '00664005402201020', '00663302100400000', '00663020100801040', '00663805400201011', '00660303500410003', '00663027800421010', '00663405820001012', '00663102400001011', '00663005400230002', '00661703100600000', '00663800100001100', '00663405400801050', '00663302400001111', '1030201698', '00663020100201150', '00663800100001090', '00662006602810000', '00663816400210022', '00663800100001130', '00663024700401030', '00663800103001010', '00660406200211010', '00663020100620000', '00663802000001010', '

In [7]:
matrl_class1.loc[matrl_class1['物料'].isin(list_duplicates1)].sort_values(by='物料')

,序号,物料,物料描述,供应商编码,供应商名称,采购员,是否进口件,物料类别,周期（天）下计划后的交货周期,在产情况,吨位区间,设备类型,备注
3599,3600,00660301600001010,吸油夹布胶管\管/◎127(五层)/700\吸油胶管(HG4-703-74)\,102264,咸宁市化建特种橡胶技术有限责任公司,杨凯荣,否,管和接头类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
3607,3608,00660301600001010,吸油夹布胶管\管/◎127(五层)/700\吸油胶管(HG4-703-74)\,102264,咸宁市化建特种橡胶技术有限责任公司,杨凯荣,否,管和接头类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
3608,3609,00660301600001020,回转泵吸油夹布胶管\管/Φ64（四层）/260\吸油胶管(HG4-703-74),102264,咸宁市化建特种橡胶技术有限责任公司,杨凯荣,否,管和接头类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
3707,3708,00660301600001020,回转泵吸油夹布胶管\管/Φ64（四层）/260\吸油胶管(HG4-703-74),102264,咸宁市化建特种橡胶技术有限责任公司,杨凯荣,否,管和接头类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
2424,2425,00660302101200000,拉板\\\00660302101200000,36098,中联恒通机械有限公司,唐子杰,否,一般结构件类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
2425,2426,00660302101200000,拉板\\\00660302101200000,36098,中联恒通机械有限公司,唐子杰,否,一般结构件类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
4501,4502,00660303500410003,右制动带\\\,31371,湘潭永达机械制造股份有限公司,周红群,否,传动件传动操纵类,12,停产,中小吨位(履320T 及以下),履带吊,NaN
4500,4501,00660303500410003,右制动带\\\,31371,湘潭永达机械制造股份有限公司,周红群,否,传动件传动操纵类,12,停产,中小吨位(履320T 及以下),履带吊,NaN
4504,4505,00660303500420001,左制动带\\\,31371,湘潭永达机械制造股份有限公司,周红群,否,传动件传动操纵类,12,停产,中小吨位(履320T 及以下),履带吊,NaN
4503,4504,00660303500420001,左制动带\\\,31371,湘潭永达机械制造股份有限公司,周红群,否,传动件传动操纵类,12,停产,中小吨位(履320T 及以下),履带吊,NaN


In [8]:
matrl_class1.loc[matrl_class1['物料'].isin(list_duplicates2)].sort_values(by='物料')

,序号,物料,物料描述,供应商编码,供应商名称,采购员,是否进口件,物料类别,周期（天）下计划后的交货周期,在产情况,吨位区间,设备类型,备注
5092,5093,00660303500401220,拉钩\\45\,100053,江西德泰隆重工股份有限公司,胡泽磊,否,管和接头类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
5182,5183,00660303500401220,拉钩\\45\,108981,长沙飞豆机械制造有限公司,胡泽磊,否,一般结构件类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
4987,4988,00663000100641031,隔套\\20\1300-010064-1031,100053,江西德泰隆重工股份有限公司,胡泽磊,否,管和接头类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
5273,5274,00663000100641031,隔套\\20\1300-010064-1031,101328,淮安市建工尼龙制品有限公司,杨凯荣,否,橡塑尼龙类,8,在产,中小吨位(履320T 及以下),履带吊,NaN
365,366,00663011630001010,卷扬过渡块\\35\1301-163000-1010,109970,建湖县恒泰机械制造有限公司,胡泽磊,否,一般结构件类,13,在产,大吨位(履350-800T),履带吊,NaN
552,553,00663011630001010,卷扬过渡块\\35\1301-163000-1010,100029,江苏鼎晟液压有限公司,黎苏明,否,液压件,8,在产,大吨位(履350-800T),履带吊,NaN
5111,5112,00663600100601090,拔销螺栓\\45\1360-010060-1090,100053,江西德泰隆重工股份有限公司,胡泽磊,否,轴承类,8,在产,大吨位(履350-800T),履带吊,NaN
5185,5186,00663600100601090,拔销螺栓\\45\1360-010060-1090,108981,长沙飞豆机械制造有限公司,胡泽磊,否,国标件,4,在产,大吨位(履350-800T),履带吊,NaN
5154,5155,00663801600001020,吸油法兰\\35\1380-160000-1020,100053,江西德泰隆重工股份有限公司,胡泽磊,否,管和接头类,8,在产,大吨位(履350-800T),履带吊,NaN
5176,5177,00663801600001020,吸油法兰\\35\1380-160000-1020,101101,盐城盛鑫源机械配件制造有限公司,胡泽磊,否,一般结构件类,13,在产,大吨位(履350-800T),履带吊,NaN


In [9]:
# 数据问题备注
matrl_class1.loc[matrl_class1['物料'].isin(list_duplicates1), '备注'] = '履带吊完全重复数据，重复两次，可以简单去重处理'
matrl_class1.loc[matrl_class1['物料'].isin(list_duplicates2), '备注'] = '履带吊冲突数据，物料号相同，但物料种类不同，需要业务纠正'

print(len(matrl_class1.loc[matrl_class1['备注']=='履带吊完全重复数据，重复两次，可以简单去重处理']))
print(len(matrl_class1.loc[matrl_class1['备注']=='履带吊冲突数据，物料号相同，但物料种类不同，需要业务纠正']))

562
14


In [10]:
# 复制一份，方便操作
matrl_class0_copy = copy.deepcopy(matrl_class0)
matrl_class1_copy = copy.deepcopy(matrl_class1)

# 经过观察，履带吊里重复的物料，不会和汽车吊交叉重复，因此这里去除履带吊重复的物料号
matrl_class1_copy.drop_duplicates(subset=['物料'], keep=False, inplace=True)
matrl_class1_copy.reset_index(drop=True, inplace=True)

In [11]:
matrl_class1_copy

,序号,物料,物料描述,供应商编码,供应商名称,采购员,是否进口件,物料类别,周期（天）下计划后的交货周期,在产情况,吨位区间,设备类型,备注
0,1,00661718300211010,板\\16Mn\,100053,江西德泰隆重工股份有限公司,胡泽磊,否,一般结构件类,14,在产,超大吨位(履800T 及以上),履带吊,NaN
1,2,00663002800601011,销轴\\40Cr\1300-280060-1011,100053,江西德泰隆重工股份有限公司,胡泽磊,否,轴承类,9,停产,中小吨位(履320T 及以下),履带吊,NaN
2,4,0066GQ20180644003,撑杆\\\,100053,江西德泰隆重工股份有限公司,胡泽磊,否,一般结构件类,13,停产,大吨位(履350-800T),履带吊,NaN
3,6,00663064900001010,轴\\40Cr\,100053,江西德泰隆重工股份有限公司,胡泽磊,否,轴承类,9,停产,中小吨位(履320T 及以下),履带吊,NaN
4,7,00663303100801050,销\\45\1330-310080-1050,100053,江西德泰隆重工股份有限公司,胡泽磊,否,轴承类,12,停产,大吨位(履350-800T),履带吊,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6060,6629,00663306600001020,弹簧销\\65Mn\1330-660000-1020,105553,常德盛达弹簧制造有限公司,黎苏明,否,弹簧类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
6061,6634,00663604100001150,弹簧插销\\钢丝II\1360-410000-1150,105553,常德盛达弹簧制造有限公司,黎苏明,否,弹簧类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
6062,6637,00663805400201080,弹簧插销\\钢丝II\1380-540020-1080,105553,常德盛达弹簧制造有限公司,黎苏明,否,弹簧类,8,停产,中小吨位(履320T 及以下),履带吊,NaN
6063,6640,00665405401201050,弹簧插销\\钢丝II\,105553,常德盛达弹簧制造有限公司,黎苏明,否,弹簧类,6,在产,中小吨位(履320T 及以下),履带吊,NaN


In [12]:
#　汽车吊和履带吊的数据合在一起，看重复或冲突情况
matrl_class = pd.concat([matrl_class0_copy, matrl_class1_copy], axis=0)
matrl_class.reset_index(drop=True, inplace=True)

print(len(matrl_class['物料']))
print(len(matrl_class['物料'].unique()))

34103
32593


In [13]:
# 汽车吊和履带吊的数据合在一起，看重复或冲突情况
# 冲突举例：物料号相同，物料类别、设备类型、交货周期、在产情况不同；
l1 = matrl_class.drop_duplicates(subset=['物料'], keep='last', inplace=False)
l2 = matrl_class.drop_duplicates(subset=['物料'], keep=False, inplace=False)
list_duplicates = set(l1['物料']) - set(l2['物料'])
print(len(list_duplicates), set(list_duplicates))

1510 {'1021400248', '1009803791', '1139900093', 'P10200000851', '1140115327', '1020404960', '1140100620', '1040200073', '00631344700401010', '1010300030', '1040100087', '202270000', '1140217092', '1020304992', '1140204729', '1050000433', '1022397580', 'P10000001660', '1021404262', '1140217090', '1000100028', '1040000311', '1019806366', '1040000295', '1000100012', '1030800003', '1019804160', '1021403925', '1010600654', '1010300008', '1081000121', '1140206279', '1140100574', '00631320221000000', '1019804119', '1020705995', '1020604226', '1040100073', '1010300295', '00631321210001020', '1021100002', '1020300013', '1040000891', '1029900664', '1039805226', '1020701018', '1029902992', '1090100453', '1021404027', '1081000069', '1031700229', '1039800904', 'P10000001868', '1009806300', '1140100708', '1010200057', '1040100004', '203090028', '1081200008', '1140100592', '1081000107', '1139800927', '1140100783', '1029905124', '1020400240', '1020304682', '1040100323', '1010300128', '1081200007', '10

In [14]:
matrl_class.loc[matrl_class['物料'].isin(list_duplicates)].sort_values(by='物料')

,序号,物料,物料描述,供应商编码,供应商名称,采购员,是否进口件,物料类别,周期（天）下计划后的交货周期,在产情况,吨位区间,设备类型,备注
18194,18195,00630710200001011,弹簧插销\\钢丝II\,105553,常德盛达弹簧制造有限公司,彭湃,否,一般结构件类,6,在产,大吨位,汽车吊,NaN
34086,6603,00630710200001011,弹簧插销\\钢丝II\,105553,常德盛达弹簧制造有限公司,黎苏明,否,弹簧类,8,停产,中小吨位(履320T 及以下),履带吊,NaN
14481,14482,00630718425000000,中心回转接头\\无\1011200019,6500,常德中联重科液压有限公司,李鹏辉,否,泵和阀类,8,在产,大吨位,汽车吊,NaN
28426,455,00630718425000000,中心回转接头\\无\1011200019,6500,常德中联重科液压有限公司,黎苏明,否,液压件,10,停产,大吨位(履350-800T),履带吊,NaN
9960,9961,00630848900001020,套\\20\1084-890000-1020,100053,江西德泰隆重工股份有限公司,贺永胜,否,一般结构件类,6,在产,大吨位,汽车吊,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31320,3593,ZCC1100HCL/32Y,零部件图册\\\,101365,长沙市雨花区浦联印刷厂,杨凯荣,否,标牌图册类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
31321,3594,ZCC1800H/32Z,零部件图册\\\,101365,长沙市雨花区浦联印刷厂,杨凯荣,否,标牌图册类,6,在产,中小吨位(履320T 及以下),履带吊,NaN
24950,24951,ZCC1800H/32Z,零部件图册\\\,101365,长沙市雨花区浦联印刷厂,喻伟,否,辅材杂件类,8,停产,中小吨位,汽车吊,NaN
24934,24935,ZCC1800HWG/23.1Z,操作手册\\\,101365,长沙市雨花区浦联印刷厂,喻伟,否,辅材杂件类,8,停产,中小吨位,汽车吊,NaN


In [15]:
matrl_class0.loc[matrl_class0['物料'].isin(list_duplicates), '备注'] = '汽车吊和履带吊的合并数据重复或冲突，冲突是否正常，是否可以纠正'
matrl_class1.loc[matrl_class1['物料'].isin(list_duplicates), '备注'] = '汽车吊和履带吊的合并数据重复或冲突，冲突是否正常，是否可以纠正'

In [16]:
# matrl_class0.to_csv('matrl_class0.csv', encoding='utf_8_sig')
# matrl_class1.to_csv('matrl_class1.csv', encoding='utf_8_sig')

In [17]:
writer = pd.ExcelWriter('save.xlsx')
matrl_class0.to_excel(writer,"Sheet0",index=0)
matrl_class1.to_excel(writer,"Sheet1",index=0)
writer.save()